In [136]:
import sqlite3
import pandas as pd
import os

"""This code is used to create the wide format of the two classifications producing the columns with
dummy variables. For the classification 1, it creates n binary columns corresponding to each detected word.
For the classification 2 and 3, it creates n binary columns corresponding. Then all the columns are stacked
to be used easily by Tableau that doesn't know how to match column to row as relationship"""

# Create a SQL connection to our SQLite database
con = sqlite3.connect("DATABASES/project.db")

cur = con.cursor()

# Classification 1 transformation

In [137]:

# Load the data into a DataFrame
regex_df = pd.read_sql_query("SELECT * from classification_1_regex", con)
match_df = pd.read_sql_query("SELECT * from classification_1_psittaciformes_or_no", con)

match_df.index = match_df.id
regex_df.index = regex_df.id

In [138]:
match_df["word"]=match_df["mapping_match"]\
    .apply(lambda x : ";".join([regex_df.at[int(_),"word"] for _ in x.split(";") if (len(x)>0)]))


In [139]:
words_df=match_df["word"].str.get_dummies(sep=";").add_prefix("word_")

In [140]:
words_df=match_df[["ad_id"]].join(words_df)
words_df

,ad_id,word_(amazona),word_african,word_africà,word_ailes,word_all-blue,word_amazon,word_amazona,word_amazone,word_ara,...,word_versicolor,word_vert,word_vicente,word_vincent,word_yaco,word_yellow-collared,word_yellow-crested,word_yellow-headed,word_yellow-naped,word_yellow-shouldered
id,,,,,,,,,,,,,,,,,,,,,
1,552731_us,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,552645_us,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,552633_us,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,552632_us,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,552631_us,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25942,1060_ph,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25943,925_ph,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25944,523_ph,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
words_df.to_sql("classification_1_analysis", con, if_exists="replace")

# Classification 2

In [142]:
class_2 = pd.read_sql_query("SELECT * from classification_2_matching_ads", con)
class_2.index = class_2.id
cites = pd.read_sql_query("SELECT * from mapping_cites", con)
cites.index = cites.id



In [143]:
class_2["species"]=class_2["ids_matching"]\
    .apply(lambda x : ";".join([cites.at[int(_),"scientific_name_cites"] for _ in x.split(";") if (x!="-1" and x!="-2" and x!="")])) #Replace all the ID's per the scientific name for each match per row
species_df=class_2["species"].str.get_dummies(sep=";").add_prefix("") #Get dummy variables to spread the species on the columns#Just if we want add a prefix later


In [144]:
species_df=class_2[["ad_id"]].join(species_df) #Add the add_id to dataframe
#Stack the dataframe to put the columns vertically 
species_df.index=species_df["ad_id"]
species_df=species_df.drop(columns=["ad_id"])
species_df=species_df.stack().to_frame().reset_index() #reindex to obtain a primary key
species_df=species_df.rename(columns={species_df.columns[0] : "ad_id", species_df.columns[1] : "species", species_df.columns[2] : "count"})
display(species_df)
species_df.to_sql("classification_2_analysis", con, if_exists="replace")

,ad_id,species,count
0,552731_us,Agapornis canus,0
1,552731_us,Agapornis fischeri,0
2,552731_us,Agapornis nigrigenis,0
3,552731_us,Agapornis personatus,0
4,552731_us,Agapornis pullarius,0
...,...,...,...
1245403,125_ph,Psephotus pulcherrimus,0
1245404,125_ph,Psittacula krameri,0
1245405,125_ph,Psittacus erithacus,0
1245406,125_ph,Pyrrhura cruentata,0


# Classification 3

In [145]:
class_3 = pd.read_sql_query("SELECT * from classification_3_matching_ads", con)
class_3.index = class_3.id
cites = pd.read_sql_query("SELECT * from mapping_cites", con)
cites.index = cites.id



In [146]:
class_3["species"]=class_3["ids_matching"]\
    .apply(lambda x : ";".join([cites.at[int(_),"scientific_name_cites"] for _ in x.split(";") if (x!="-1" and x!="-2" and x!="")])) #Replace all the ID's per the scientific name for each match per row
species_df=class_3["species"].str.get_dummies(sep=";").add_prefix("") #Get dummy variables to spread the species on the columns#Just if we want add a prefix later


In [147]:
species_df=class_3[["ad_id"]].join(species_df) #Add the add_id to dataframe
#Stack the dataframe to put the columns vertically 
species_df.index=species_df["ad_id"]
species_df=species_df.drop(columns=["ad_id"])
species_df=species_df.stack().to_frame().reset_index() #reindex to obtain a primary key
species_df=species_df.rename(columns={species_df.columns[0] : "ad_id", species_df.columns[1] : "species", species_df.columns[2] : "count"})
display(species_df)
species_df.to_sql("classification_3_analysis", con, if_exists="replace")

,ad_id,species,count
0,552731_us,Agapornis fischeri,0
1,552731_us,Agapornis personatus,0
2,552731_us,Agapornis roseicollis,0
3,552731_us,Amazona arausiaca,0
4,552731_us,Amazona auropalliata,0
...,...,...,...
1011889,125_ph,Psephotus dissimilis,0
1011890,125_ph,Psephotus pulcherrimus,0
1011891,125_ph,Psittacula krameri,0
1011892,125_ph,Psittacus erithacus,0


In [148]:
# Be sure to close the connection
con.close()